In [ ]:
# Dependencies
!pip install praw
!pip install nltk
!pip install transformers
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
# Imports

#get data
import praw
import webbrowser
import os

#data manipulation
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

#data pre processing
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from operator import index
from transformers import pipeline

#model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from google.colab import drive

#save model and tokenizer
from tensorflow.keras.models import load_model
import pickle

In [ ]:
#credential removed for safety

# client_id = "id"
# client_secret = "secret"
# user_agent = "user"
# redirect_uri = "http://localhost:8080"

In [ ]:
# reddit object
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
    redirect_uri=redirect_uri
)

auth_url = reddit.auth.url(scopes=["identity", "read"], state="random_string") # "identity" para acessar informações do usuário, "read" para ler dados públicos
print(f"Abra este URL no seu navegador para autorizar o acesso: {auth_url}")

webbrowser.open(auth_url) # Isso abrirá o URL no seu navegador


Abra este URL no seu navegador para autorizar o acesso: https://www.reddit.com/api/v1/authorize?client_id=lS299rUWY4RtAIaeoFoBbA&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity+read&state=random_string


False

In [ ]:
authorization_code = "iiQ98BPletKkukTmWJ8H1og4xyOD-w"

In [ ]:
try:
  reddit.auth.authorize(authorization_code)
  print("Autorização bem-sucedida!")
except Exception as e:
  print(f"Erro na autorização: {e}")

Autorização bem-sucedida!


In [ ]:
subreddit_name = "climatechange"
search_terms = [
    "climate change",
    "global warming",
    "climate crisis",
    "climate emergency",
    "environment",
    "sustainability",
    "ecology",
    "climate",
    "planet",
    "atmosphere",
    "greenhouse effect",
    "carbon emissions",
    "greenhouse gases",
    "fossil fuels",
    "oil",
    "coal",
    "natural gas",
    "deforestation",
    "wildfires",
    "agriculture methane emissions",
    "industrialization",
    "overconsumption",
    "extreme weather events",
    "drought",
    "flooding",
    "storm",
    "hurricane",
    "cyclone",
    "heatwave",
    "forest fires",
    "sea level rise",
    "melting glaciers",
    "biodiversity loss",
    "species extinction",
    "ocean acidification",
    "food security",
    "water scarcity",
    "climate migration",
    "health impacts climate change",
    "renewable energy",
    "solar power",
    "wind power",
    "hydroelectric power",
    "geothermal energy",
    "biomass energy",
    "energy transition",
    "decarbonization",
    "carbon neutrality",
    "circular economy",
    "energy efficiency",
    "carbon capture",
    "reforestation",
    "sustainable agriculture",
    "electric mobility",
    "green hydrogen",
    "Paris Agreement",
    "COP",
    "COP26",
    "COP27",
    "COP28",
    "COP29",
    "COP30",
    "climate policies",
    "environmental legislation",
    "climate activism",
    "climate justice",
    "climate action",
    "government climate change",
    "companies sustainability",
    "NGOs environment",
    "IPCC reports",
    "climate skepticism",
    "climate denial",
    "climate misinformation",
    "climate conspiracy theories",
    "#ClimateChange",
    "#GlobalWarming",
    "#ClimateCrisis",
    "#ClimateEmergency",
    "#Environment",
    "#Sustainability",
    "#Ecology",
    "#Climate",
    "#ActOnClimate",
    "#SaveOurPlanet",
    "#GreenNewDeal",
    "#RenewableEnergy",
    "#NetZero",
    "#FridaysForFuture",
    "#ExtinctionRebellion",
    "#ClimateJustice"
]
limit = 10  # amount limit

collected_posts = []

try:
    subreddit = reddit.subreddit(subreddit_name)
    for term in search_terms:
        print(f"Searching for: '{term}' in r/{subreddit_name}")
        for submission in subreddit.search(query=term, sort="relevance", limit=limit):
            post_data = {
                "id": submission.id,
                "title": submission.title,
                "author": str(submission.author),
                "url": submission.url,
                "created_utc": submission.created_utc,
                "selftext": submission.selftext, # post content (if its a text post)
                "subreddit": subreddit_name,
                "search_term": term
            }
            collected_posts.append(post_data)
        print(f"Collected {len(collected_posts)} posts until now.")

except Exception as e:
    print(f"Error during collect: {e}")

print("\nCollected data:")
for post in collected_posts:
    print(f"ID: {post['id']}, Título: {post['title']}")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Searching for: 'climate change' in r/climatechange


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Collected 10 posts until now.
Searching for: 'global warming' in r/climatechange


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

Collected 847 posts until now.
Searching for: '#FridaysForFuture' in r/climatechange


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Collected 850 posts until now.
Searching for: '#ExtinctionRebellion' in r/climatechange


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Collected 859 posts until now.
Searching for: '#ClimateJustice' in r/climatechange
Collected 869 posts until now.

Collected data:
ID: 1i0psw5, Título: We’ve Crossed a Key Threshold for Climate Change. There’s No Going Back Now.
ID: 1c9qpn1, Título: As someone who is uneducated on the topic of climate change, is it really exaggerated in media?
ID: 1j7wsf3, Título: Are we actually making progress on climate change, or are we just fooling ourselves?
ID: 1gk8y4k, Título: Trump would be an "Extinction-Level Event" for the Planet, Turbocharging Climate Change. Vote Accordingly.
ID: 1hhbjh1, Título: Scientists say sprinkling diamond dust into the sky could offset almost all of climate change so far — but it'll cost $175 trillion
ID: 1hkm5gu, Título: Venezuela is the first country to lose all of its glaciers due to climate change
ID: 1gcn54d, Título: Why do some people deny climate change so passionately? 
ID: 1j87bml, Título: Secretary Hegseth says the DOD does not do 'climate change crap'
I

In [ ]:
# convert dictionary into a dataframe
df = pd.DataFrame(collected_posts)

In [ ]:
df.head()

,id,title,author,url,created_utc,selftext,subreddit,search_term
0,1i0psw5,We’ve Crossed a Key Threshold for Climate Chan...,Slate,https://slate.com/technology/2025/01/hottest-y...,1.736805e+09,,climatechange,climate change
1,1c9qpn1,As someone who is uneducated on the topic of c...,Apart-Management2989,https://www.reddit.com/r/climatechange/comment...,1.713729e+09,I'm genuinely curious as someone who only has ...,climatechange,climate change
2,1j7wsf3,Are we actually making progress on climate cha...,Significant-Lemon596,https://www.reddit.com/r/climatechange/comment...,1.741607e+09,Are we actually making enough progress on clim...,climatechange,climate change
3,1gk8y4k,"Trump would be an ""Extinction-Level Event"" for...",LudovicoSpecs,https://www.juancole.com/2024/11/extinction-tu...,1.730820e+09,,climatechange,climate change
4,1hhbjh1,Scientists say sprinkling diamond dust into th...,Chilango615,https://www.livescience.com/planet-earth/clima...,1.734555e+09,,climatechange,climate change


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           869 non-null    object 
 1   title        869 non-null    object 
 2   author       869 non-null    object 
 3   url          869 non-null    object 
 4   created_utc  869 non-null    float64
 5   selftext     869 non-null    object 
 6   subreddit    869 non-null    object 
 7   search_term  869 non-null    object 
dtypes: float64(1), object(7)
memory usage: 54.4+ KB


In [ ]:
print(df['search_term'].value_counts())

search_term
climate change         10
global warming         10
climate crisis         10
climate emergency      10
environment            10
                       ..
hydroelectric power     6
overconsumption         5
#FridaysForFuture       3
NGOs environment        2
COP30                   2
Name: count, Length: 91, dtype: int64


In [ ]:
# drop duplicates posts
df.drop_duplicates(subset='id', keep='first', inplace=True) # Remove duplicados com base no ID único do post

In [ ]:
# fillna in "selftext", avoiding "combination problems" --> "tile" + "selftext"
df['selftext'].fillna('', inplace=True)

<ipython-input-14-e25f983724e6>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['selftext'].fillna('', inplace=True)


In [ ]:
df['combined_text_title&selftext'] = df['title'] + '' + df['selftext']

In [ ]:
df['combined_text_title&selftext'] = df['combined_text_title&selftext'].str.lower()

In [ ]:
# removing characters different from: numbers, letters or blanks
df['combined_text_title&selftext'] = df['combined_text_title&selftext'].apply(lambda x: re.sub(r'[^a-z0-9\s]]', '', x))

In [ ]:
# words tokenize
nltk.download('punkt_tab')
df['tokens'] = df['combined_text_title&selftext'].apply(word_tokenize)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# removing "stopwords" (word without significant meaning)
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
df['tokens_cleaned'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# lemmatizing tokens (reducing words into their base form)
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
df['lemmatized_tokens'] = df['tokens_cleaned'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])

[nltk_data] Downloading package wordnet to /root/nltk_data...


# EMBENDDINGS

In [ ]:
# downloading embeddings files
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d embeddings

embeddings_index = {}
with open('embeddings/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype = 'float32')
        embeddings_index[word] = coefs

print(f'Found {len(embeddings_index)} word vectors.')

--2025-05-15 19:56:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-05-15 19:56:18--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-15 19:56:19--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# create a vocabulary from lematized tokens
tokenizer = Tokenizer(num_words = None) # maintain all the words
tokenizer.fit_on_texts(df['lemmatized_tokens'].apply(lambda x: ' '.join(x)))
word_index = tokenizer.word_index
print(f'Vocabulary length: {len(word_index)}')

# define max length of sequence for padding
MAX_SEQUENCE_LENGTH = max(df['lemmatized_tokens'].apply(len))

# create embeddings matrix
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # not found words in Glove ll be vectors of zeros
        embedding_matrix[i] = embedding_vector

print(f'Shape of embedding matrix: {embedding_matrix.shape}')

Vocabulary length: 8234
Shape of embedding matrix: (8235, 100)


In [ ]:
# convert tokens on number sequences and applying padding (garantee all the sequences to have the same size)
sequences = tokenizer.texts_to_sequences(df['lemmatized_tokens'].apply(lambda x: ' '.join(x)))
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

print(f'Shape of padded sequences: {padded_sequences.shape}')

Shape of padded sequences: (659, 1124)


# LSTM Model

In [ ]:
# LSTM model
VOCAB_SIZE = len(tokenizer.word_index) + 1
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 1124

model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │       823,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 823,500 (3.14 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 823,500 (3.14 MB)

# Labeling database

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# labeling

#loading sentiment-analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def get_sentiment(text):
  try:
    # truncating text for max length the model can work with
    max_length = sentiment_pipeline.tokenizer.model_max_length - 2
    truncated_text = sentiment_pipeline.tokenizer.decode(
        sentiment_pipeline.tokenizer.encode(text, truncation=True, max_length=max_length),
        skip_special_tokens=True
    )
    result = sentiment_pipeline(truncated_text)[0]
    return result['label'], result['score']
  except Exception as e:
    print(f"Error classifying text: {e}")
    return None, None

tqdm.pandas()

sentiment_results = df['combined_text_title&selftext'].progress_apply(get_sentiment)

df[['sentiment_label', 'sentiment_score']]= pd.DataFrame(sentiment_results.tolist(), index=df.index)

print(df[['title', 'combined_text_title&selftext', 'sentiment_label', 'sentiment_score']].head())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
100%|██████████| 659/659 [03:36<00:00,  3.04it/s]

                                               title  \
0  We’ve Crossed a Key Threshold for Climate Chan...   
1  As someone who is uneducated on the topic of c...   
2  Are we actually making progress on climate cha...   
3  Trump would be an "Extinction-Level Event" for...   
4  Scientists say sprinkling diamond dust into th...   

                        combined_text_title&selftext sentiment_label  \
0  we’ve crossed a key threshold for climate chan...        NEGATIVE   
1  as someone who is uneducated on the topic of c...        NEGATIVE   
2  are we actually making progress on climate cha...        NEGATIVE   
3  trump would be an "extinction-level event" for...        NEGATIVE   
4  scientists say sprinkling diamond dust into th...        NEGATIVE   

   sentiment_score  
0         0.980067  
1         0.995722  
2         0.999625  
3         0.976432  
4         0.771831  


In [ ]:
# converting labels to binarie format
sentiment_mapping = {'POSITIVE':1, 'NEGATIVE':0}
df['sentiment_label_encoded'] = df['sentiment_label'].map(sentiment_mapping)



#getting index of remaining lines
# getting index of remaining lines
# The issue was here. `df.index` was holding old index values,
# even after rows were dropped from the DataFrame.
# Resetting the index ensures the index values are in sync with the DataFrame's current state.
indexes = df.reset_index(drop=True).index

# converting into numpy arrays for Keras
X = np.array(padded_sequences)[indexes]
y = np.array(df['sentiment_label_encoded'])

# splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (527, 1124)
Shape of X_test: (132, 1124)
Shape of y_train: (527,)
Shape of y_test: (132,)


# LSTM Model Training

In [ ]:
epochs = 10
batch_size = 32

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test))

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.7334 - loss: 0.6664 - val_accuracy: 0.7273 - val_loss: 0.6256
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 85s 4s/step - accuracy: 0.7274 - loss: 0.6296 - val_accuracy: 0.7273 - val_loss: 0.6089
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.7200 - loss: 0.6027 - val_accuracy: 0.7273 - val_loss: 0.5868
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.7495 - loss: 0.5643 - val_accuracy: 0.7273 - val_loss: 0.5893
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.7366 - loss: 0.5802 - val_accuracy: 0.7273 - val_loss: 0.5860
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.7347 - loss: 0.5775 - val_accuracy: 0.7273 - val_loss: 0.5861
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.7297 - loss: 0.5886 - val_accuracy: 0.7273 - val_loss: 0.5862
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.7220 - loss: 0.5940 - val_accuracy: 0.7273 - val_loss:

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss : {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7081 - loss: 0.6038
Loss : 0.5867
Accuracy: 0.7273


In [ ]:
#saving data frame

#naming directory
data_path = 'data'

if not os.path.exists(data_path):
    os.makedirs(data_path)
    print(f"Directory '{data_path}' created successfully.")
else:
    print(f"Directory '{data_path}' already exists.")

#saving
df.to_csv('data/df_labeled.csv', index=False)
print('DataFrame saved successfully.')

Directory 'data' created successfully.
DataFrame saved successfully.


In [ ]:
# saving model and tokenizer
model_tokenizer_Path = 'model'

if not os.path.exists(model_tokenizer_Path):
    os.makedirs(model_tokenizer_Path)
    print(f"Directory '{model_tokenizer_Path}' created successfully.")
else:
    print(f"Directory '{model_tokenizer_Path}' already exists.")

#model
try:
    model.save('model/sentiment_model.keras')
    print("Model saved successfully.")
except Exception as e:
    print(f"Error saving the model: {e}")

#tokenizer
try:
    with open('model/tokenizer.pickle', 'wb') as handle:
      pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
      print('Tokenizer saved successfully.')
except Exception as e:
    print(f"Error saving the tokenizer: {e}")

Directory 'model' created successfully.
Model saved successfully.
Tokenizer saved successfully.
